In [1]:
default_vals_dict = {
  "payment_type": 5,
  "RatecodeID": 1
}

yellow_taxi_trips_df = spark.read.format("delta").load("/mnt/datalake/yellow_taxi_trips")

yellow_taxi_trips_df = yellow_taxi_trips_df \
                        .filter(("passenger_count > 0 AND trip_distance > 0")) \
                        \
                        .dropna(subset=["PULocationID", "DOLocationID"]) \
                        \
                        .fillna(default_vals_dict) \
                        \
                        .drop_duplicates() \
                        \
                        .filter("tpep_pickup_datetime >= '2018-12-01' AND tpep_dropoff_datetime <= '2018-12-31'" )

In [2]:
from pyspark.sql.functions import col, year, month, dayofmonth, unix_timestamp, round, when

# Applying transformations

print("Starting transformation on Yellow Taxi data")

yellow_taxi_trips_df = yellow_taxi_trips_df \
                             .select( \
                              col('VendorID').alias('vendor_id'), \
                              col('tpep_pickup_datetime').alias('pickup_time'), \
                              col('tpep_dropoff_datetime').alias('dropoff_time'), \
                              col('trip_distance'), \
                              col('PULocationID').alias('pickup_loc'), \
                              col('DOLocationID').alias('dropoff_loc'), \
                              col('RatecodeID').alias('rate_code_id'), \
                              col('total_amount'), \
                              col('payment_type') \
                              ) \
                              \
                             .withColumn("trip_year", year('pickup_time')) \
                             .withColumn("trip_month", month('pickup_time')) \
                             .withColumn("trip_day", dayofmonth('pickup_time')) \
                              \
                              .withColumn("trip_duration", \
                                          round((unix_timestamp('dropoff_time') - unix_timestamp('pickup_time')) / 60) \
                                         ) \
                              \
                              .withColumn( \
                                          'trip_type', \
                                                      when(
                                                      col('rate_code_id') == 6, 'shared_trip'
                                                      )\
                                                      .when(
                                                      col('rate_code_id') == 1000, 'shared_trip' # There is no rate_code_id value of 1000. Have included for "switch case" example
                                                      )
                                                      .otherwise('solo_trip')
                                         ) \
                              .drop('rate_code_id')

print("Applied transformations on Yellow Taxi data")

Starting transformation on Yellow Taxi data
Applied transformations on Yellow Taxi data

In [3]:
yellow_taxi_trips_df.createOrReplaceGlobalTempView('fact_yellow_taxi_trip_data')

print("Saved Yellow Taxi fact as a global temp view")

Saved Yellow Taxi fact as a global temp view

In [4]:
print("Starting to save Yellow Taxi dataframe as a fact and unmanaged table")

yellow_taxi_trips_df.write \
                  .mode('overwrite') \
                  .format('delta') \
                  .option('path', '/mnt/datalake/dimensional_model/facts/yellow_taxi_fact.delta') \
                  .saveAsTable('taxi_service_warehouse.fact_yellow_taxi_trips_data')

print("Saved Yellow Taxi dataframe as a fact and unmanaged table")

Starting to save Yellow Taxi dataframe as a fact and unmanaged table
Saved Yellow Taxi dataframe as a fact and unmanaged table

In [5]:
dbutils.notebook.exit("Success")